# Working with Radio Sounding Data
This is a short tutorial how to work with sounding data using [MetPy](https://unidata.github.io/MetPy/latest/index.html#).

In [ ]:
# first some packages and functions are imported
import pandas as pd
import numpy as np
# functions for downloading the data
from soundings import (create_url, from_day_time, check_todays_sounding, 
                       write_csvfile, download_sounding, create_dataframe)

## Downloading the current sounding data
The data is provided on this [website](http://weather.uwyo.edu/upperair/sounding.html). For this step MetPy is not necessary.

In [ ]:
# This functions downloads the data of the last sounding and creates a pandas dataframe:
df = download_sounding()
df

The names of the columns stand for:

- pres: - Pressure in hPa
- hght: - Geopotential Height in m
- temp: - Temperature in °C
- dwpt: - Dewpoint Temperature in °C
- relh: - Relative Humidity in %
- mixr: - Mixing Ratio in gram/kilogram
- drct: - Wind Direction in degrees true
- speed: - Wind Speed in knot
- thta: - Potential Temperature in K
- thte: - Equivalent Potential Temperature in K
- thtv: - Virtual Potential Temperature in K


If MetPy is not yet installed on your PC: you can download it like it is described here: https://unidata.github.io/MetPy/latest/installguide.html

In [ ]:
# for using MetPy we need a few more packages and functions
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import Hodograph, SkewT
from metpy.units import units

MetPy gives us the opportunity to assign units to physical quantities. This is mandatory for using some of its functions. The corresponding package is called 'units'. There has to be paid attention when using it for temperature, see [here](https://pint.readthedocs.io/en/latest/nonmult.html).

In [ ]:
# assigning the appropriate units to the physical quantities
p = df['pres'].values * units.hPa
h = df['hght'].values * units.meters
t = df['temp'].values * units.celsius
dt = df['dwpt'].values * units.celsius
rh = df['relh'].values * units.percent
mr = df['mixr'].values * units.gram / units.kilogram
dr = df['drct'].values * units.degree
ws = df['speed'].values * units.knot
thta = df['thta'].values * units.kelvin
thte = df['thte'].values * units.kelvin
thtv = df['thtv'].values * units.kelvin

In [ ]:
# calculation of the windspeed in x- and y-direction with the implemented functions
df['u_wind'], df['v_wind'] = mpcalc.wind_components(df['speed'],
                                                    np.deg2rad(df['drct']))
wind_speed = df['speed'].values * units.knots
wind_dir = df['drct'].values * units.degrees
u, v = mpcalc.wind_components(wind_speed, wind_dir)

## Calculations with MetPy

[Here](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html) you can find the supported calculations one can do with MetPy. In the following we have a look at a few of them.
(The examples are mostly taken from the [MetPy-Tutorial](https://unidata.github.io/MetPy/latest/tutorials/upperair_soundings.html#sphx-glr-tutorials-upperair-soundings-py).)

### Basic Skew-T Plotting

In [ ]:
# Create a new figure. The dimensions here give a good aspect ratio
fig = plt.figure(figsize=(9, 18))
skew = SkewT(fig)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(p, t, 'r', linewidth=2)
skew.plot(p, dt, 'g', linewidth=2)
skew.plot_barbs(p, u, v)

# Show the plot
plt.show()
# The plot will be saved in your current folder as skewplot.pdf.
fig.savefig('skewplot.pdf');

### LCL (lifted condensation level) and parcel profile
[Documentation of lcl](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.lcl.html#metpy.calc.lcl)

[Documentation of parcel profile](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.parcel_profile.html#metpy.calc.parcel_profile)

In [ ]:
# Calculate the LCL
# use first entry in pressure, temperature and dewpoint temperature
lcl_pressure, lcl_temperature = mpcalc.lcl(p[0], t[0], dt[0])

print(lcl_pressure, lcl_temperature)

# Calculate the parcel profile.
parcel_prof = mpcalc.parcel_profile(p, t[0], dt[0]).to('degC')

### Advanced Skew-T Plotting with an added Hodograph

In [ ]:
# Create a new figure. The dimensions here give a good aspect ratio
fig = plt.figure(figsize=(14, 14))
skew = SkewT(fig, rotation=30)

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(p, t, 'r')
skew.plot(p, dt, 'g')
skew.plot_barbs(p, u, v)
skew.ax.set_ylim(1000, 100)
skew.ax.set_xlim(-40, 60)

# Plot LCL temperature as black dot
skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')

# Plot the parcel profile as a black line
skew.plot(p, parcel_prof, 'k', linewidth=2)

# Shade areas of CAPE and CIN
skew.shade_cin(p, t, parcel_prof)
skew.shade_cape(p, t, parcel_prof)

# Plot a zero degree isotherm
skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

# Add the relevant special lines
skew.plot_dry_adiabats()
skew.plot_moist_adiabats()
skew.plot_mixing_lines()

# Create a hodograph
# Create an inset axes object that is 40% width and height of the
# figure and put it in the upper right hand corner.
ax_hod = inset_axes(skew.ax, '40%', '40%', loc=1)
h = Hodograph(ax_hod, component_range=80.)
h.add_grid(increment=20)
h.plot_colormapped(u, v, wind_speed)  # Plot a line colored by wind speed

# Show the plot
plt.show()
fig.savefig('skewplot_hodograph.pdf');

### Lapse-Rate

[Documentation of dry lapse rate](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.dry_lapse.html#metpy.calc.dry_lapse)

In [ ]:
lr_d = mpcalc.dry_lapse(p,t[0])

In [ ]:
plt.plot(lr_d.to('degC'));